In [18]:
import pymongo
import pandas as pd
from pymongo import MongoClient
import os
from dotenv import load_dotenv

load_dotenv()
ATLAS_KEY = os.getenv("ATLAS_KEY")
client = MongoClient(ATLAS_KEY)
db = client["imbd-scrap"]
movies_collection = db["movies"]

### 1) Quel est le film le plus long ?

In [2]:
longest_movie = movies_collection.find_one(sort=[('time', -1)], projection={'title': 1, 'time': 1, '_id': 0})
print(longest_movie)

{'title': 'Gone with the Wind', 'time': 238}


### 2) Quels sont les 5 films les mieux notés ?

In [19]:
def get_top_movies(n=5):
    top_movies = movies_collection.find({}, {"_id": 0, "title": 1, "score": 1}).sort("score", pymongo.DESCENDING).limit(n)
    return pd.DataFrame(top_movies)

get_top_movies(5)

,title,score
0,The Shawshank Redemption,9.3
1,The Godfather,9.2
2,The Godfather Part II,9.0
3,12 Angry Men,9.0
4,Schindler's List,9.0


Le code ci-dessous affiche les 5 films les mieux notés ou plus en cas d'égalité des scores

In [16]:
top_movies = movies_collection.find({}, {'title': 1, 'score': 1}).sort('score', pymongo.DESCENDING)

# Recupérer le 5ème score
fifth_score = top_movies[4]['score']

# Filtrer les films avec un score supérieur ou égal au 5ème score
top_movies = [movie for movie in top_movies if movie['score'] >= fifth_score]

for movie in top_movies:
    print(movie['title'], "   Score: ",movie['score'])

The Shawshank Redemption    Score:  9.3
The Godfather    Score:  9.2
The Lord of the Rings: The Return of the King    Score:  9.0
Schindler's List    Score:  9.0
12 Angry Men    Score:  9.0
The Godfather Part II    Score:  9.0
The Dark Knight    Score:  9.0


### 3) Dans combien de films a joué Morgan Freeman ? Tom Cruise ?

In [4]:
actor = "Morgan Freeman"
count = movies_collection.count_documents({"stars": actor})
print(f"{actor} a joué dans {count} films")

Morgan Freeman a joué dans 4 films


In [5]:
actor = "Tom Cruise"
count = movies_collection.count_documents({"stars": actor})
print(f"{actor} a joué dans {count} films")

Tom Cruise a joué dans 1 films


### 4) Quels sont les 3 meilleurs films d’horreur ? Dramatique ? Comique ?

In [6]:
list_genre = ["Horror", "Drama", "Comedy"]

for genre in list_genre:
    print(f"Les 3 meilleurs films de la catégorie '{genre}' sont :")
    movies = movies_collection.find({'genre': genre}).sort([('score', -1)]).limit(3)
    for movie in movies:
        print(movie['title'], movie['score'])
    print('\n')


Les 3 meilleurs films de la catégorie 'Horror' sont :
Alien 8.5
Psycho 8.5
The Shining 8.4


Les 3 meilleurs films de la catégorie 'Drama' sont :
The Shawshank Redemption 9.3
The Godfather 9.2
Schindler's List 9.0


Les 3 meilleurs films de la catégorie 'Comedy' sont :
Life Is Beautiful 8.6
The Intouchables 8.5
Modern Times 8.5




Pareil que précedemment, le code ci-dessous gère les égalités : 

In [17]:
list_genre = ["Horror", "Drama", "Comedy"]

for genre in list_genre:
    print(f"Les 3 meilleurs films de la catégorie '{genre}' sont :")
    top_movies = list(movies_collection.find({'genre': genre}).sort([('score', -1)]).limit(3))
    if len(top_movies) > 0:
        top_score = top_movies[-1]['score']
        additional_movies = list(movies_collection.find({'genre': genre, 'score': top_score}).sort([('score', -1)]).skip(3))
        top_movies.extend(additional_movies)
        for i, movie in enumerate(top_movies):
            print(f"{i+1}. {movie['title']} - Score: {movie['score']}")
    else:
        print("Aucun film trouvé dans cette catégorie")
    print('\n')


Les 3 meilleurs films de la catégorie 'Horror' sont :
1. Alien - Score: 8.5
2. Psycho - Score: 8.5
3. The Shining - Score: 8.4


Les 3 meilleurs films de la catégorie 'Drama' sont :
1. The Shawshank Redemption - Score: 9.3
2. The Godfather - Score: 9.2
3. Schindler's List - Score: 9.0
4. The Godfather Part II - Score: 9.0
5. The Dark Knight - Score: 9.0


Les 3 meilleurs films de la catégorie 'Comedy' sont :
1. Life Is Beautiful - Score: 8.6
2. The Intouchables - Score: 8.5
3. Modern Times - Score: 8.5
4. Back to the Future - Score: 8.5




### 5) Parmi les 100 films les mieux notés, quel pourcentage sont américains ? Français ?

In [7]:
# Récupération des 100 films les mieux notés
top_100_movies = movies_collection.find().sort('score', pymongo.DESCENDING).limit(100)

# Initialisation des compteurs pour les films américains et français
nb_american_movies = 0
nb_french_movies = 0

# Parcours de la liste des 100 films pour compter les films américains et français
for movie in top_100_movies:
    if 'United States' in movie['country']:
        nb_american_movies += 1
    if 'France' in movie['country']:
        nb_french_movies += 1


# Affichage des résultats
print(f"Parmi les 100 films les mieux notés, {nb_american_movies}% sont américains et {nb_french_movies}% sont français.")


Parmi les 100 films les mieux notés, 76% sont américains et 10% sont français.


### 6) Quel est la durée moyenne d’un film en fonction du genre ?

In [8]:
pipeline = [
    {"$group": {"_id": "$genre", "avg_duration": {"$avg": "$time"}}},
    {"$project": {"genre": "$_id", "avg_duration": 1, "_id": 0}}
]

results = movies_collection.aggregate(pipeline)

for result in results:
    print(result)


{'avg_duration': 109.0, 'genre': ['Horror', 'Mystery', 'Sci-Fi']}
{'avg_duration': 110.0, 'genre': ['Animation', 'Action', 'Adventure']}
{'avg_duration': 133.4, 'genre': ['Action', 'Adventure', 'Fantasy']}
{'avg_duration': 89.0, 'genre': ['Drama', 'Family', 'Sport']}
{'avg_duration': 128.0, 'genre': ['Mystery', 'Romance', 'Thriller']}
{'avg_duration': 168.5, 'genre': ['Action', 'Drama']}
{'avg_duration': 145.0, 'genre': ['Drama', 'Romance', 'Thriller']}
{'avg_duration': 140.625, 'genre': ['Drama', 'War']}
{'avg_duration': 95.0, 'genre': ['Comedy', 'War']}
{'avg_duration': 92.0, 'genre': ['Animation', 'Comedy', 'Drama']}
{'avg_duration': 129.0, 'genre': ['Drama', 'Mystery', 'Romance']}
{'avg_duration': 45.0, 'genre': ['Action', 'Comedy', 'Romance']}
{'avg_duration': 130.75, 'genre': ['Drama', 'Mystery', 'Thriller']}
{'avg_duration': 121.0, 'genre': ['Mystery', 'Thriller']}
{'avg_duration': 138.5, 'genre': ['Drama', 'Family']}
{'avg_duration': 128.66666666666666, 'genre': ['Action', 'Dra

In [9]:
import pandas as pd

genres = movies_collection.distinct('genre')
result = {}

for genre in genres:
    count = movies_collection.count_documents({'genre': genre})
    total_duration = sum(movie['time'] for movie in movies_collection.find({'genre': genre}))
    avg_duration = total_duration / count
    result[genre] = round(avg_duration, 2)

df = pd.DataFrame(list(result.items()), columns=['genre', 'durée moyenne (min)'])
print(df)


        genre  durée moyenne (min)
0      Action               133.98
1   Adventure               130.12
2   Animation               100.96
3   Biography               145.50
4      Comedy               105.60
5       Crime               132.50
6       Drama               134.58
7      Family               114.54
8     Fantasy               123.57
9   Film-Noir               110.00
10    History               152.11
11     Horror               120.60
12      Music               134.25
13    Musical               103.00
14    Mystery               124.61
15    Romance               118.48
16     Sci-Fi               131.70
17      Sport               128.50
18   Thriller               123.88
19        War               140.43
20    Western               153.86


In [20]:
def count_movies_by_actor(actor_name: str):
    '''
    Compte le nombre de films dans lesquels un acteur a joué.

    Args:
        actor_name (str): Nom de l'acteur.

    Returns:
        str: Chaîne de caractères indiquant le nombre de films dans lesquels l'acteur a 
    '''
    count = movies_collection.count_documents({"stars": {"$regex": actor_name, "$options": "i"}})
    return f"{actor_name} a joué dans {count} films"

print(type(count))

<class 'int'>
